
# Use scikit-learn to recognize hand-written digits with `ibm-watson-machine-learning`

This notebook contains steps and code to demonstrate how to persist and deploy locally trained scikit-learn model in Watson Machine Learning Service. This notebook contains steps and code to work with [ibm-watson-machine-learning](https://pypi.python.org/pypi/ibm-watson-machine-learning) library available in PyPI repository. This notebook introduces commands for getting model and training data, persisting model, deploying model, scoring it, updating the model and redeploying it.

Some familiarity with Python is helpful. This notebook runs on Python. It uses the ibm-watson-machine-learning package.

## Learning goals

The learning goals of this notebook are:

-  Train sklearn model.
-  Persist trained model in Watson Machine Learning repository.
-  Deploy model for online scoring using client library.
-  Score sample records using client library.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)
2.	[Explore data and create scikit-learn model](#train)
3.	[Persist externally created scikit model](#persistence)
4.	[Deploy and score in a Cloud](#scoring)
5.  [Clean up](#cleanup)
6.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Create a <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance (a free plan is offered and information about how to create the instance can be found <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-service-instance.html?context=analytics" target="_blank" rel="noopener no referrer">here</a>).

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud. You need to provide platform `api_key` and instance `location`.

You can use [IBM Cloud CLI](https://cloud.ibm.com/docs/cli/index.html) to retrieve platform API Key and instance location.

API Key can be generated in the following way:
```
ibmcloud login
ibmcloud iam api-key-create API_KEY_NAME
```

In result, get the value of `api_key` from the output.


Location of your WML instance can be retrieved in the following way:
```
ibmcloud login --apikey API_KEY -a https://cloud.ibm.com
ibmcloud resource service-instance WML_INSTANCE_NAME
```

In result, get the value of `location` from the output.

**Tip**: Your `Cloud API key` can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users). From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. Give your key a name and click **Create**, then copy the created key and paste it below. You can also get a service specific url by going to the [**Endpoint URLs** section of the Watson Machine Learning docs](https://cloud.ibm.com/apidocs/machine-learning).  You can check your instance location in your  <a href="https://console.ng.bluemix.net/catalog/services/ibm-watson-machine-learning/" target="_blank" rel="noopener no referrer">Watson Machine Learning (WML) Service</a> instance details.

You can also get service specific apikey by going to the [**Service IDs** section of the Cloud Console](https://cloud.ibm.com/iam/serviceids).  From that page, click **Create**, then copy the created key and paste it below.

**Action**: Enter your `api_key` and `location` in the following cell.

In [ ]:
api_key = 'PASTE YOUR PLATFORM API KEY HERE'
location = 'PASTE YOUR INSTANCE LOCATION HERE'

In [ ]:
wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}

### Install and import the `ibm-watson-machine-learning` package
**Note:** `ibm-watson-machine-learning` documentation can be found <a href="http://ibm-wml-api-pyclient.mybluemix.net/" target="_blank" rel="noopener no referrer">here</a>.

In [ ]:
!pip install -U ibm-watson-machine-learning

In [ ]:
from ibm_watson_machine_learning import APIClient

client = APIClient(wml_credentials)

### Working with spaces

First, you need to create a space that will be used for your work. If you do not have space already created, you can use [Deployment Spaces Dashboard](https://dataplatform.cloud.ibm.com/ml-runtime/spaces?context=cpdaas) to create one.

- Click New Deployment Space
- Create an empty space
- Select Cloud Object Storage
- Select Watson Machine Learning instance and press Create
- Copy `space_id` and paste it below

**Tip**: You can also use SDK to prepare the space for your work. More information can be found [here](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Space%20management.ipynb).

**Action**: Assign space ID below

You can use `list` method to print all existing spaces.

In [ ]:
client.spaces.list(limit=10)

In [ ]:
space_id = 'PASTE YOUR SPACE ID HERE'

To be able to interact with all resources available in Watson Machine Learning, you need to set the **space** which you will be using.

In [ ]:
client.set.default_space(space_id)

<a id="train"></a>
## 2. Explore data and create a scikit-learn model
In this section, you will prepare and train handwritten digits model using scikit-learn library.

### 2.1 Explore data

As a first step, you will load the data from scikit-learn sample datasets and perform a basic exploration.

In [ ]:
import sklearn
from sklearn import datasets

digits = datasets.load_digits()

Loaded toy dataset consists of 8x8 pixels images of hand-written digits.

Let's display first digit data and label using **data** and **target**.

In [ ]:
print(digits.data[0])

In [ ]:
print(digits.data[0].reshape((8, 8)))

In [ ]:
digits.target[0]

In the next step, you will count data examples.

In [ ]:
samples_count = len(digits.images)
print("Number of samples: " + str(samples_count))

### 2.2. Create a scikit-learn model

**Prepare data**

In this step, you'll split your data into three datasets:
- train
- test
- score

In [ ]:
train_data = digits.data[: int(0.7*samples_count)]
train_labels = digits.target[: int(0.7*samples_count)]

test_data = digits.data[int(0.7*samples_count): int(0.9*samples_count)]
test_labels = digits.target[int(0.7*samples_count): int(0.9*samples_count)]

score_data = digits.data[int(0.9*samples_count): ]

print("Number of training records: " + str(len(train_data)))
print("Number of testing records : " + str(len(test_data)))
print("Number of scoring records : " + str(len(score_data)))

**Create pipeline**

Next, you'll create scikit-learn pipeline.

In ths step, you will import scikit-learn machine learning packages that will be needed in next cells.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import svm, metrics

Standardize features by removing the mean and scaling to unit variance.

In [ ]:
scaler = preprocessing.StandardScaler()

Next, define estimators you want to use for classification. Support Vector Machines (SVM) with radial basis function as kernel is used in the following example.

In [ ]:
clf = svm.SVC(kernel='rbf')

Let's build the pipeline now. This pipeline consists of transformer and an estimator.

In [ ]:
pipeline = Pipeline([('scaler', scaler), ('svc', clf)])

**Train model**

Now, you can train your SVM model by using the previously defined **pipeline** and **train data**.

In [ ]:
model = pipeline.fit(train_data, train_labels)

**Evaluate model**

You can check your **model quality** now. To evaluate the model, use **test data**.

In [ ]:
predicted = model.predict(test_data)

print("Evaluation report: \n\n%s" % metrics.classification_report(test_labels, predicted))

You can tune your model now to achieve better accuracy. For simplicity of this example tuning section is omitted.

<a id="persistence"></a>
## 3. Persist locally created scikit-learn model

In this section, you will learn how to store your model in Watson Machine Learning repository by using the IBM Watson Machine Learning SDK.

### 3.1: Publish model

#### Publish model in Watson Machine Learning repository on Cloud.

Define model name, autor name and email.

In [ ]:
sofware_spec_uid = client.software_specifications.get_id_by_name("default_py3.8")

In [ ]:
print(sofware_spec_uid)

In [ ]:
metadata = {
            client.repository.ModelMetaNames.NAME: 'Scikit model',
            client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
            client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: sofware_spec_uid
}

published_model = client.repository.store_model(
    model=model,
    meta_props=metadata,
    training_data=train_data,
    training_target=train_labels)

### 3.2: Get model details

In [ ]:
import json

published_model_uid = client.repository.get_model_uid(published_model)
model_details = client.repository.get_details(published_model_uid)
print(json.dumps(model_details, indent=2))

### 3.3 Get all models

In [ ]:
models_details = client.repository.list_models()

<a id="scoring"></a>
## 4. Deploy and score in a Cloud

In this section you will learn how to create online scoring and to score a new data record by using the IBM Watson Machine Learning SDK.

### 4.1: Create model deployment

#### Create online deployment for published model

In [ ]:
metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of scikit model",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create(published_model_uid, meta_props=metadata)

**Note**: Here we use the deployment url saved in the `published_model` object. In next section, we show how to retrieve the deployment url from a Watson Machine Learning instance.

In [ ]:
deployment_uid = client.deployments.get_uid(created_deployment)

Now you can print an online scoring endpoint. 

In [ ]:
scoring_endpoint = client.deployments.get_scoring_href(created_deployment)
print(scoring_endpoint)

You can also list existing deployments.

In [ ]:
client.deployments.list()

### 4.2: Get deployment details

In [ ]:
client.deployments.get_details(deployment_uid)

### 4.3: Score

You can use the following method to do test scoring request against deployed model.

**Action**: Prepare scoring payload with records to score.

In [ ]:
score_0 = list(score_data[0])
score_1 = list(score_data[1])

In [ ]:
scoring_payload = {"input_data": [{"values": [score_0, score_1]}]}

Use ``client.deployments.score()`` method to run scoring.

In [ ]:
predictions = client.deployments.score(deployment_uid, scoring_payload)

In [ ]:
print(json.dumps(predictions, indent=2))

<a id="cleanup"></a>
## 5. Clean up

If you want to clean up all created assets:
- experiments
- trainings
- pipelines
- model definitions
- models
- functions
- deployments

please follow up this sample [notebook](https://github.com/IBM/watson-machine-learning-samples/blob/master/cloud/notebooks/python_sdk/instance-management/Machine%20Learning%20artifacts%20management.ipynb).

<a id="summary"></a>
## 6. Summary and next steps

 You successfully completed this notebook! You learned how to use scikit-learn machine learning as well as Watson Machine Learning for model creation and deployment. Check out our _[Online Documentation](https://dataplatform.cloud.ibm.com/docs/content/wsj/getting-started/welcome-main.html?context=analytics?pos=2)_ for more samples, tutorials, documentation, how-tos, and blog posts. 

### Authors

**Daniel Ryszka**, Software Engineer

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.